# UDP통신
    
1. socket 이란?
   > IP와 포트를 활용 : 특정 포트에 연결되어 데이터를 보내거나 받고자 할 때  
   > <font color='red'>서버</font> : 서비스 주체로 클라이언트로부터 메시지 받고 응답한다.  
   > <font color='blue'>클라이언트</font> : 서비스 사용자  
2. UDP서버와 클라이언트를 만들어 보자
   
   

## 1. 사용중인 컴퓨터의 IP 확인

In [ ]:
!ipconfig 

## 2. UDP 클라이언트
1. 주소 체계(family)
   > AF_INET : IPv4  
   > AF_INET6 : IPv6
2. 타입(type)
   > SOCK_STREAM : TCP  
   > SOCK_DGRAM : UDP

In [ ]:
#통신 모듈: UDP클라이언트(노트북)
import socket
import time
import threading
global gmsg  # main, thread 전역 변수
global tellomsg # Tello 상태 정보
global run_flag # thread 종료 변수
#주소 설정
tello_addr = ('0.0.0.0',38889) # Tello드론 38889
notebook_addr = ('0.0.0.0',38890) # 노트북 38890

# 메시지 송수신
def send(message,delay):
    try:
        sock.sendto(message.encode(),tello_addr)
        print("[notebook]메시지:" + message)
    except Exception as e:
        print("local 에러 메시지:" + str(e))
    #지연
    time.sleep(delay)        
# tello -> notebook 메시지 받기(recvfrom)
def receive():
    global gmsg
    global tellomsg
    global run_flag
    while run_flag:
        try:
            response, ip_addr = sock.recvfrom(1024)            
            gmsg = "[tello]: %s, IP: %s" % (response.decode(encoding='UTF-8'), str(ip_addr))            
            print(gmsg)            
        except Exception as e:
            print("local receive 에러 메시지:" + str(e))
            break
            
# 로컬(notebook) 통신포트 바인딩
sock = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
sock.bind(notebook_addr)

# 노트북으로부터 메시지 받기 쓰레드 생성 및 시작
run_flag = True
recvThread = threading.Thread(target=receive)
recvThread.daemon = True
recvThread.start()            

# 통신 시작
while True:
    try:
        print('###############################\n■ 명령어입력: command->takeoff->land\n■ 프로그램 종료: quit\n###############################\n')
        message = input()
        if 'quit' in message:
            print('프로그램 종료')
            run_flag = False
            recvThread.join()
            sock.close()
            break
        else:
            send(message,1)
    except KeyboardInterrupt as e:
        run_flag = False
        recvThread.join()
        sock.close()
        break